In [8]:
import pandas as pd
import re

# Setting part
# will later be stored in settings file that can be read in

# Define index variable and variable that stay fixed for each case
# These should be at the START of the datafile
index_col = ["Uuid"]
fixed_col_names = ["Date", "Weight_tracking", "Country"]

# define prefixes to be reshaped
"""
prefix is every variant of variable names that come *before* the variablegroup ID (ID has to be numeric)
example: 'QBVDETAILS_bv1ch' -> prefix is 'QBVDETAILS_bv', ID is '1', suffix is 'ch'

A prefix group is a set of prefixes that behaves identical

A general prefix are prefixes that apply to all groups and should be summarized in the same variable
"""

prefix_groups = 2  # currently 2 groups: ads & moodboards
general_prefix = []

# define FIRST prefixes for each group
"""
prefix is every variant of variable names that come *before* the variablegroup ID (ID has to be numeric)
example: 'QADDETAILS_ad1ch' -> prefix is 'QADDETAILS_ad', ID is '1', suffix is 'ch'
only the prefixes need to be noted here. IDs and suffixes will be detected automatically
Needs to be a list of lists: One list per prefix group
(For Aldi there are 2 prefix groups: Ads & Moodboards)
"""
col_prefix = [["QFOODETAILS_foo", "VQ_foobarfoo"],
              ["VQ_foobarbarfoos"]]

# define method for determining new col names
"""
currently only the method "first" is supported
this takes the *first* name of each variable name for the new column names
example: values QADDETAILS_ad1ch, QADDETAILS_ad2ch, QADDETAILS_ad3ch... will all be found under QADDETAILS_ad1ch
I personally strongly dislike this method, as this leads to confusing column names,
but it is how the master files are currently setup
"""
new_col_names_method = "first"

if new_col_names_method != "first":
    print(
        "Unknown new_col_names_method!\n Currently only 'first' is supported.")

wide = pd.read_excel("test.xlsx")
wide.head()

,Uuid,Date,Weight_tracking,Country,QFOODETAILS_foo1nm,QFOODETAILS_foo1br,QFOODETAILS_foo1cid,QFOODETAILS_foo1url,QFOODETAILS_foo1ch,VQ_foobarfoo1r,...,VQ_foobarbarfoosBARf11,VQ_foobarbarfoosBARf12,VQ_foobarbarfoosBARf13,VQ_foobarbarfoosBARf14,VQ_foobarbarfoosBARf15,VQ_foobarbarfoosBARf16,VQ_foobarbarfoosBARf17,VQ_foobarbarfoosBARf18,VQ_foobarbarfoosBARf19,VQ_foobarbarfoosBARf20
0,6z36r8mzkgrsmat1,9/27/2021,0.809053,Süd,tukmiKqSod,NVfk,lYYU,kSED,TV,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,thbqg2kdasv4vgw1,9/27/2021,0.775600,Nord,tukmiKqSod,NVfk,lYYU,kSED,TV,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sg7u3zd8749x8h67,9/27/2021,0.775600,Nord,tukmiKqSod,NVfk,lYYU,kSED,TV,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3qk1kd336hkpuhqg,9/27/2021,0.832696,Süd,tukmiKqSod,NVfk,lYYU,kSED,TV,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hxdg0hpp95wdadxk,9/27/2021,0.878053,Süd,tukmiKqSod,NVfk,lYYU,kSED,TV,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
long = pd.melt(wide[:2], index_col, value_vars = ["QFOODETAILS_foo1nm","QFOODETAILS_foo2nm"], value_name="QFOODETAILS_foo1nm")
long.sort_values("Uuid", inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
long.head()

,Uuid,variable,QFOODETAILS_foo1nm
0,6z36r8mzkgrsmat1,QFOODETAILS_foo1nm,tukmiKqSod
2,6z36r8mzkgrsmat1,QFOODETAILS_foo2nm,RKPh
1,thbqg2kdasv4vgw1,QFOODETAILS_foo1nm,tukmiKqSod
3,thbqg2kdasv4vgw1,QFOODETAILS_foo2nm,RKPh


In [11]:
value_cols_current_gr = []
cols = list(wide.columns)
for index_var in index_col: cols.remove(index_var)
for fixed_var in fixed_col_names : cols.remove(fixed_var)

# loop that goes through each prefix group
for group in col_prefix:
    # for each prefix, find the appropriate new column name
    # and all old columns that need to be reshaped for this prefix
    for prefix in group:
        pattern = prefix + "1(\D|$)"
        r = re.compile(pattern)
        print(r)
        new_col_names = list(filter(r.match, cols))
        print(new_col_names)

re.compile('QFOODETAILS_foo1(\\D|$)')
['QFOODETAILS_foo1nm', 'QFOODETAILS_foo1br', 'QFOODETAILS_foo1cid', 'QFOODETAILS_foo1url', 'QFOODETAILS_foo1ch']
re.compile('VQ_foobarfoo1(\\D|$)')
['VQ_foobarfoo1r', 'VQ_foobarfoo1s1', 'VQ_foobarfoo1s2', 'VQ_foobarfoo1s3', 'VQ_foobarfoo1s4', 'VQ_foobarfoo1s5', 'VQ_foobarfoo1s6', 'VQ_foobarfoo1s7', 'VQ_foobarfoo1s8', 'VQ_foobarfoo1s9', 'VQ_foobarfoo1s10', 'VQ_foobarfoo1f1', 'VQ_foobarfoo1f2', 'VQ_foobarfoo1f3', 'VQ_foobarfoo1f4', 'VQ_foobarfoo1f5', 'VQ_foobarfoo1f6', 'VQ_foobarfoo1f7', 'VQ_foobarfoo1f8', 'VQ_foobarfoo1f9', 'VQ_foobarfoo1f10', 'VQ_foobarfoo1f11', 'VQ_foobarfoo1f12', 'VQ_foobarfoo1f13', 'VQ_foobarfoo1f14', 'VQ_foobarfoo1f15', 'VQ_foobarfoo1f16', 'VQ_foobarfoo1f17', 'VQ_foobarfoo1f18', 'VQ_foobarfoo1f19', 'VQ_foobarfoo1f20']
re.compile('VQ_foobarbarfoos1(\\D|$)')
[]


In [12]:
cols

['QFOODETAILS_foo1nm',
 'QFOODETAILS_foo1br',
 'QFOODETAILS_foo1cid',
 'QFOODETAILS_foo1url',
 'QFOODETAILS_foo1ch',
 'VQ_foobarfoo1r',
 'VQ_foobarfoo1s1',
 'VQ_foobarfoo1s2',
 'VQ_foobarfoo1s3',
 'VQ_foobarfoo1s4',
 'VQ_foobarfoo1s5',
 'VQ_foobarfoo1s6',
 'VQ_foobarfoo1s7',
 'VQ_foobarfoo1s8',
 'VQ_foobarfoo1s9',
 'VQ_foobarfoo1s10',
 'VQ_foobarfoo1f1',
 'VQ_foobarfoo1f2',
 'VQ_foobarfoo1f3',
 'VQ_foobarfoo1f4',
 'VQ_foobarfoo1f5',
 'VQ_foobarfoo1f6',
 'VQ_foobarfoo1f7',
 'VQ_foobarfoo1f8',
 'VQ_foobarfoo1f9',
 'VQ_foobarfoo1f10',
 'VQ_foobarfoo1f11',
 'VQ_foobarfoo1f12',
 'VQ_foobarfoo1f13',
 'VQ_foobarfoo1f14',
 'VQ_foobarfoo1f15',
 'VQ_foobarfoo1f16',
 'VQ_foobarfoo1f17',
 'VQ_foobarfoo1f18',
 'VQ_foobarfoo1f19',
 'VQ_foobarfoo1f20',
 'QFOODETAILS_foo2nm',
 'QFOODETAILS_foo2br',
 'QFOODETAILS_foo2cid',
 'QFOODETAILS_foo2url',
 'QFOODETAILS_foo2ch',
 'VQ_foobarfoo2r',
 'VQ_foobarfoo2s1',
 'VQ_foobarfoo2s2',
 'VQ_foobarfoo2s3',
 'VQ_foobarfoo2s4',
 'VQ_foobarfoo2s5',
 'VQ_foobarfoo2s